In [106]:
import pandas as pd # for data manipulation
import numpy as np # for data manipulation
from sklearn.linear_model import LinearRegression # for building a linear regression model
from pyearth import Earth # for building a MARS model
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
from icecream import ic

In [107]:
df = pd.read_csv('mobile_price_train.csv', encoding='utf-8')
# Print dataframe
# df["price"] = df.apply((lambda price: np.random.uniform(200,400) if df["price_range"] == 0 else (np.random.uniform(400, 600) if df["price_range"] == 1 else (np.random.uniform(600,1000) if df["price_range"] else np.random.uniform(1000, 1500)))), axis=1)
val = df["price_range"].value_counts()
df.sort_values("price_range", inplace=True)
price_float = []
for i in range (0, 4):
    price_float = [*price_float, *np.random.uniform(1500/4*i, 1500/4*(i+1), size=500)]
df["price_float"] = price_float
df


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range,price_float
1838,720,1,0.9,1,12,1,14,0.8,165,4,...,1234,1086,14,7,3,1,1,0,0,117.175298
1762,808,1,0.5,1,3,0,46,0.5,105,8,...,529,1082,15,5,10,1,0,1,0,146.067399
419,914,1,0.9,0,2,0,4,0.8,100,2,...,1495,808,11,7,5,1,1,0,0,296.970165
846,1489,0,2.4,1,8,0,32,0.6,200,1,...,812,776,7,6,8,0,0,0,0,272.503589
845,1907,0,2.0,0,13,0,55,0.9,105,7,...,1513,286,14,4,17,1,0,1,0,50.926672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,1046,1,2.8,1,0,1,58,0.2,100,8,...,1040,3863,17,12,9,1,1,1,3,1348.872624
982,1035,0,0.6,1,2,1,44,0.5,99,3,...,1960,3506,14,5,12,1,1,0,3,1180.261543
987,1688,0,2.5,0,0,1,21,0.2,170,7,...,864,3984,17,14,8,1,1,1,3,1146.471215
956,862,0,2.3,0,9,1,6,0.1,107,8,...,1977,3458,9,4,9,1,1,1,3,1484.504904


In [108]:
# Create a scatter plot
fig = px.scatter(df, x=df['ram'], y=df['price_float'],
                 opacity=0.8, color_discrete_sequence=['black'])

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black')

# Set figure title
fig.update_layout(title_text="Scatter Plot")

# Update marker size
fig.update_traces(marker=dict(size=3))

fig.show()


In [109]:
# --- Select variables to use in the two models ---
# Note, we need X to be a 2D array, hence reshape
X=df['ram'].values.reshape(-1,1)
y=df['price_float'].values

# --- Define and fit the two models ---
model1 = LinearRegression()
model2 = Earth(max_terms=500, max_degree=1) # note, terms in brackets are the hyperparameters

LR = model1.fit(X, y)
MARS = model2.fit(X, y)

# --- Print model summary ---
# LR
print("Simple Linear Regression Model")
print("--------------------------------------")
print("Intercept: ", LR.intercept_)
print("Slope: ", LR.coef_)

print("")
print("<><><><><><><><><><><><><><><><><><><>")
print("")

# MARS
print(MARS.summary())


Simple Linear Regression Model
--------------------------------------
Intercept:  -9.237203130716921
Slope:  [0.35834723]

<><><><><><><><><><><><><><><><><><><>

Earth Model
-------------------------------------
Basis Function  Pruned  Coefficient  
-------------------------------------
(Intercept)     No      117.662      
h(x0-3426)      No      -0.363979    
h(3426-x0)      Yes     None         
h(x0-824)       No      0.249646     
h(824-x0)       Yes     None         
x0              No      0.148332     
-------------------------------------
MSE: 39270.7181, GCV: 39606.6593, RSQ: 0.7952, GRSQ: 0.7937


C:\Users\madex\anaconda3\envs\sklearn-env\lib\site-packages\pyearth\earth.py:813: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

C:\Users\madex\anaconda3\envs\sklearn-env\lib\site-packages\pyearth\earth.py:1066: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



In [110]:
# ----- Create data for Linear Regression line -----
# Create 20 evenly spaced points from smallest X to largest X
x_range = np.linspace(X.min(), X.max(), 20)

# Predict y values for our set of X values
y_range = model1.predict(x_range.reshape(-1, 1))



# ----- Create data for MARS model line -----
# Select a few points including the 3 major ones: min, max and hinge
x_mars = np.array([X.min(), 1000, 1146.33, 2000, 3000, 4000, 5000, 6000, X.max()])

# Predict y values for our set of X values
y_mars = model2.predict(x_mars.reshape(-1, 1))



# ----- Create a scatter plot -----
fig = px.scatter(df, x=df['ram'], y=df['price_float'],
                 opacity=0.8, color_discrete_sequence=['black'])

# Add a best-fit line
fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Linear Regression', line=dict(color='limegreen')))
fig.add_traces(go.Scatter(x=x_mars, y=y_mars, name='MARS', line=dict(color='red')))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                 showline=True, linewidth=1, linecolor='black')

# Set figure title
fig.update_layout(title_text="Linear Regression vs. MARS")

# Update marker size
fig.update_traces(marker=dict(size=3))

fig.show()


In [116]:
# Create a 3D scatter plot
fig = px.scatter_3d(df, x=df['pc'], y=df['int_memory'], z=df['price_range'],
                 opacity=0.8, color_discrete_sequence=['black'])

# Set figure title
fig.update_layout(title_text="Scatter 3D Plot",
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey')))

# Update marker size
fig.update_traces(marker=dict(size=3))

fig.show()

In [112]:
# ----- Select variables that we want to use in a model -----
# Note, X in this case is already a 2D array, hence no reshape
X=df[['pc','int_memory']]
y=df['price_float'].values

# ----- Define and fit the two models -----
model1 = LinearRegression()
model2 = Earth()

LR = model1.fit(X, y)
MARS = model2.fit(X, y)

# ----- Print model summary -----
# LR
print("Simple Linear Regression Model")
print("--------------------------------------")
print("Intercept: ", LR.intercept_)
print("Slope: ", LR.coef_)

print("")
print("<><><><><><><><><><><><><><><><><><><>")
print("")

# MARS
print(MARS.summary())


Simple Linear Regression Model
--------------------------------------
Intercept:  701.1765702086535
Slope:  [2.42691619 0.83396184]

<><><><><><><><><><><><><><><><><><><>

Earth Model
--------------------------------------
Basis Function   Pruned  Coefficient  
--------------------------------------
(Intercept)      No      751.969      
int_memory       Yes     None         
h(int_memory-0)  Yes     None         
h(0-int_memory)  Yes     None         
pc               Yes     None         
h(pc-0)          Yes     None         
h(0-pc)          Yes     None         
--------------------------------------
MSE: 191768.4706, GCV: 191960.3830, RSQ: -0.0000, GRSQ: -0.0000


C:\Users\madex\anaconda3\envs\sklearn-env\lib\site-packages\pyearth\earth.py:813: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

C:\Users\madex\anaconda3\envs\sklearn-env\lib\site-packages\pyearth\earth.py:1066: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



In [113]:
# Increments between points in a meshgrid
mesh_size = 5

# Identify min and max values for input variables
x_min, x_max = X['pc'].min(), X['pc'].max()
y_min, y_max = X['int_memory'].min(), X['int_memory'].max()

# Return evenly spaced values based on a range between min and max
xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)

# Create a meshgrid
xx, yy = np.meshgrid(xrange, yrange)

# Predict using LR model
pred_LR = model1.predict(np.c_[xx.ravel(), yy.ravel()])
pred_LR = pred_LR.reshape(xx.shape)

# Predict using MARS model
pred_MARS = model2.predict(np.c_[xx.ravel(), yy.ravel()])
pred_MARS = pred_MARS.reshape(xx.shape)

# Note, .ravel() flattens the array to a 1D array,
# then np.c_ takes elements from flattened xx and yy arrays and puts them together,
# this creates the right shape required for model input

# prediction array that is created by the model output is a 1D array,
# we need to reshape it to be the same shape as xx or yy to be able to display it on a graph


C:\Users\madex\anaconda3\envs\sklearn-env\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



In [114]:
# Create a 3D scatter plot with predictions
fig = px.scatter_3d(df, x=df['pc'], y=df['int_memory'], z=df['price_float'],
                 opacity=0.8, color_discrete_sequence=['black'])

# Set figure title and colors
fig.update_layout(title_text="Scatter 3D Plot with Linear Regression Prediction Surface",
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey')))
# Update marker size
fig.update_traces(marker=dict(size=3))

# Add prediction plane
fig.add_traces(go.Surface(x=xrange, y=yrange, z=pred_LR, name='LR'))

fig.show()


In [115]:
# Create a 3D scatter plot with predictions
fig = px.scatter_3d(df, x=df['pc'], y=df['int_memory'], z=df['price_float'],
                 opacity=0.8, color_discrete_sequence=['black'])

# Set figure title and colors
fig.update_layout(title_text="Scatter 3D Plot with MARS Prediction Surface",
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey')))
# Update marker size
fig.update_traces(marker=dict(size=3))

# Add prediction plane
fig.add_traces(go.Surface(x=xrange, y=yrange, z=pred_MARS, name='MARS', ))
                          #colorscale=px.colors.sequential.Viridis))

fig.show()